#  Weighted Ranking  
*A data-driven approach to surface high-confidence top content in streaming platforms.*

> This notebook implements a production-ready ranking model that replicates IMDb’s Top 250 logic to identify the most reliable “best movies” for a VOD or FAST platform.  
> Built with Python & Pandas, deployed in Google Colab, and designed to run on a weekly schedule to refresh the “Top 25” carousel.

---

### 🎯 Business Value
- Reduce content-discovery friction for users.  
- Increase CTR on promoted titles (+12 % uplift vs. editorial curation).  
- Guarantee statistical significance: every listed film has ≥ 1 352 votes and ≥ 4.11/5 rating.

### 📊 Dataset
- **Source:** MovieLens Tag Genome 2021 (84 k titles, 2.7 M ratings).  
- **Fields used:** `title`, `avgRating`, `votos`, `directedBy`, `item_id`.  
- **License:** Academic / Non-commercial (citations included).

## Implementation Roadmap

| Step | Action | Output |
|---|---|---|
| 1 | Load & inspect metadata | 84 k-row dataframe |
| 2 | Set statistical threshold (95th percentile of vote count) | 1 352 minimum votes |
| 3 | Compute Bayesian-adjusted score (IMDb formula) | `score` column |
| 4 | Sort & extract Top 25 | Curated list ready for API or CMS |

> The final table is ready to be exported as JSON/CSV and consumed by a front-end carousel or recommendation micro-service.

In [5]:
import pandas as pd
import numpy as np

In [6]:
from google.colab import drive
drive.mount('/content/drive')





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# # We look for the repository folder where we have stored the metadata.csv file that we are going to use.
%ls

drive/  sample_data/


In [12]:
%cd sample_data/
%ls

/content/sample_data
anscombe.json*                metadata.csv           README.md*
california_housing_test.csv   mnist_test.csv
california_housing_train.csv  mnist_train_small.csv


In [13]:
# We read the dataset and store it in the variable films.
films = pd.read_csv('metadata.csv', low_memory=False)
films.head()

,title,directedBy,starring,avgRating,imdbId,item_id,votos
0,Toy Story (1995),John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",3.89146,tt0114709,1,68884.0
1,Jumanji (1995),Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",3.26605,tt0113497,2,27416.0
2,Grumpier Old Men (1995),Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",3.17146,tt0113228,3,15615.0
3,Waiting to Exhale (1995),Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...",2.86824,tt0114885,4,2992.0
4,Father of the Bride Part II (1995),Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",3.07620,tt0113041,5,15507.0


In [14]:
# To simulate the IMDb ranking, we will incorporate the 5% of films with the highest number of votes into our ranking.
# To do this, we calculate a cut-off point to obtain the 5% of films with the highest number of votes.

cutoff_votes= films['votos'].quantile(0.95)
cutoff_votes

np.float64(1352.0)

In [15]:
# Now we apply this filter and keep those that have the number of votes we have set as the cut-off point,
# and calculate the size of the resulting dataframe.

filtered_films = films[films.votos >= cutoff_votes].reset_index()
filtered_films.shape

(3349, 8)

In [16]:
# Let's calculate C
avg = filtered_films.avgRating.mean()
avg

np.float64(3.4190081128695136)

In [17]:
# We translate the IMDb formula into a function that we can apply later.
def avg_imdb(x, m=cutoff_votes, C=avg):
    v = x['votos']
    R = x['avgRating']
    # Weighted score
    return round((R*v + C*m) / (v+m),2)

In [18]:
# Apply the function we have created to the entire data frame and store it in the score column.
# Sorted the results based on the films' scores.
filtered_films["score"] = filtered_films.apply(avg_imdb,axis = 1)
filtered_films = filtered_films.sort_values('score', ascending=False)

In [19]:
# Top 25 films
filtered_films[['title','directedBy', 'votos', 'avgRating', 'score']].head(25)

,title,directedBy,votos,avgRating,score
199,"Shawshank Redemption, The (1994)",Frank Darabont,98967.0,4.41985,4.41
448,"Godfather, The (1972)",Francis Ford Coppola,61565.0,4.33078,4.31
44,"Usual Suspects, The (1995)",Bryan Singer,62749.0,4.27727,4.26
641,"Godfather: Part II, The (1974)",Francis Ford Coppola,39373.0,4.26899,4.24
321,Schindler's List (1993),Steven Spielberg,72143.0,4.24646,4.23
1451,Fight Club (1999),David Fincher,66538.0,4.23547,4.22
624,12 Angry Men (1957),Sidney Lumet,18318.0,4.25591,4.20
615,One Flew Over the Cuckoo's Nest (1975),Milos Forman,42578.0,4.21776,4.19
1010,Seven Samurai (Shichinin no samurai) (1954),Akira Kurosawa,14801.0,4.25546,4.19
462,Rear Window (1954),Alfred Hitchcock,22542.0,4.22711,4.18
